'''for index, i in data.iterrows():
    
    room=i['Cabin']
    print(index, room)
    if not pd.isnull(room):  
        data.at[index, 'Deck']=room[0]
data=data.drop(['Cabin'], axis=1)'''

PRE-PROCESSING

In [1]:
import pandas as pd
import numpy as np
import re
data=pd.read_csv('Rents.csv', sep=',', encoding='iso8859_4')
print(data.head(5))
print(data.shape)
nums=[0]*58548
data['Posti']=nums
data['Kaupunginosa']=nums
data['Kaupunki']=nums

# creates new rows for postcode, city and district
for index, row in data.iterrows():
    full=row['Postinumero']
    post=full[0:6]
    full=full[6:]
    i=0
    for chr in full:
        if chr=='(':
            paikka=full[0:i]
            kaupunki=full[i+1:-1]
            break
        i+=1   
    post = re.sub(r'\s', '', post)
    kaupunki = re.sub(r'\s', '', kaupunki)
    paikka = re.sub(r'\s', '', paikka)

    # add nans for missing points
    if row['Neliövuokra (eur/m2)'] == ".": 
        data.at[index, 'Neliövuokra (eur/m2)']= np.nan
    else:
        data.at[index, 'Neliövuokra (eur/m2)']= float(row['Neliövuokra (eur/m2)'])
    
    data.at[index, 'Posti']= post
    data.at[index, 'Kaupunginosa']= paikka
    data.at[index, 'Kaupunki']= kaupunki 
    #print(post, paikka, kaupunki, "je")
data=data.drop(['Postinumero'], axis=1)

# important cities
remainers = ['Helsinki', 'Vantaa', 'Espoo', 'Turku', 'Tampere']
data=data.where(data['Kaupunki'].isin(remainers)).dropna(subset=['Kaupunki']) 

# data['Neliövuokra (eur/m2)']=data['Neliövuokra (eur/m2)'].astype(float) #not needed anymore?
print(data.head(3))


  Vuosineljännes                                        Postinumero Huoneluku  \
0         2015Q1  00100 Helsinki Keskusta - Etu-Töölö   (Helsinki )    Yksiöt   
1         2015Q1  00100 Helsinki Keskusta - Etu-Töölö   (Helsinki )   Kaksiot   
2         2015Q1  00100 Helsinki Keskusta - Etu-Töölö   (Helsinki )  Kolmiot+   
3         2015Q1                      00120 Punavuori   (Helsinki )    Yksiöt   
4         2015Q1                      00120 Punavuori   (Helsinki )   Kaksiot   

  Lukumäärä Neliövuokra (eur/m2)  
0       452                24.26  
1       243                18.36  
2        34                13.09  
3        95                21.52  
4        52                14.77  
(58548, 5)


/tmp/ipykernel_11697/2222452167.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00100' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Posti']= post
/tmp/ipykernel_11697/2222452167.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'HelsinkiKeskusta-Etu-Töölö' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Kaupungin_osa']= paikka
/tmp/ipykernel_11697/2222452167.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Helsinki' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Kaupunki']= kaupunki


  Vuosineljännes Huoneluku Lukumäärä  Neliövuokra (eur/m2)  Posti  \
0         2015Q1    Yksiöt       452                 24.26  00100   
1         2015Q1   Kaksiot       243                 18.36  00100   
2         2015Q1  Kolmiot+        34                 13.09  00100   

                Kaupungin_osa  Kaupunki  
0  HelsinkiKeskusta-Etu-Töölö  Helsinki  
1  HelsinkiKeskusta-Etu-Töölö  Helsinki  
2  HelsinkiKeskusta-Etu-Töölö  Helsinki  


In [6]:
huoneet=['Yksiöt', 'Kaksiot', 'Kolmiot+']
setit=set()
uutuus = pd.DataFrame().reindex_like(data)
#print(uutuus.head())
i =0

for index, row in data.iterrows():
    post=row['Posti']
    #print(post)
    if post not in setit:
        setit.add(post)
        for huone in huoneet:
            pala=data.loc[(data['Huoneluku']==huone)& (data['Posti']==post)]
            #print(pala.head(5))
            pala['Neliövuokra (eur/m2)']=pala['Neliövuokra (eur/m2)'].ffill() #fills nan
            #print(pala.head(5))
            for ind, row in pala.iterrows():
                uutuus.loc[i]=row
                i+=1
uutuus=uutuus.dropna(subset=['Kaupunki'])                

asd


/tmp/ipykernel_11697/3577020003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pala['Neliövuokra (eur/m2)']=pala['Neliövuokra (eur/m2)'].ffill() #fills nan
/tmp/ipykernel_11697/3577020003.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015Q1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i]=row
/tmp/ipykernel_11697/3577020003.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yksiöt' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  uutuus.loc[i]=row
/tmp/ipykernel_11697/3577020003.py:18: FutureW

ARIMA

# Prices 1/2/3


In [13]:
import pandas as pd
import numpy as np
import re

def Data_wrangle(data : pd.DataFrame):
    # creates new rows for postcode, city and district
    nums=[0]*len(data)
    data['Posti']=nums
    data['Kaupunginosa']=nums
    data['Kaupunki']=nums
    for index, row in data.iterrows():
        full=row['Postinumero']
        post=full[0:6]
        full=full[6:]
        i=0
        for chr in full:
            if chr=='(':
                paikka=full[0:i]
                kaupunki=full[i+1:-1]
                break
            i+=1   
        post = re.sub(r'\s', '', post)
        kaupunki = re.sub(r'\s', '', kaupunki)
        paikka = re.sub(r'\s', '', paikka)
        
        # add nans for missing points
        if row['Neliöhinta (EUR/m2)'] == "." or row['Neliöhinta (EUR/m2)'] == "...": 
            data.at[index, 'Neliöhinta (EUR/m2)']= np.nan
        else:
            data.at[index, 'Neliöhinta (EUR/m2)']= float(row['Neliöhinta (EUR/m2)'])

        data.at[index, 'Posti']= post
        data.at[index, 'Kaupunginosa']= paikka
        data.at[index, 'Kaupunki']= kaupunki 
        #print(post, paikka, kaupunki, "je")
    data=data.drop(['Postinumero'], axis=1)

    # important cities
    remainers = ['Helsinki', 'Vantaa', 'Espoo', 'Turku', 'Tampere']
    data=data.where(data['Kaupunki'].isin(remainers)).dropna(subset=['Kaupunki']) 
    return data

data_prices = pd.concat([Data_wrangle(pd.read_csv(x, sep=',', encoding='iso8859_4')) for x in ['Prices 1bed.csv', 'Prices 2bed.csv', 'Prices 3bed.csv']]) #reads all 3 and wrangles them and 

print(data_prices.head(3))

/tmp/ipykernel_11697/4258628934.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00100' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Posti']= post
/tmp/ipykernel_11697/4258628934.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Helsinkikeskusta-Etu-Töölö' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Kaupunginosa']= paikka
/tmp/ipykernel_11697/4258628934.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Helsinki' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[index, 'Kaupunki']= kaupunki
/tmp/ipykernel_11697/4258628934.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and w

  Vuosineljännes         Talotyyppi Neliöhinta (EUR/m2)  \
0         2009Q1  Kerrostalo yksiöt              4751.0   
1         2009Q1  Kerrostalo yksiöt              4561.0   
2         2009Q1  Kerrostalo yksiöt                 NaN   

  Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen  Posti  \
0                                                 36                00100   
1                                                  8                00120   
2                                                  1                00130   

                 Kaupunginosa  Kaupunki  
0  Helsinkikeskusta-Etu-Töölö  Helsinki  
1         Punavuori-Bulevardi  Helsinki  
2             Kaartinkaupunki  Helsinki  


In [17]:
# Change Talotyyppi -> Huoneluku, Kauppojen lukumäärä, vara... -> Lukumäärä
data_prices.rename(columns={'Talotyyppi': 'Huoneluku', 'Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen' : 'Lukumäärä'}, inplace=True)

# values in Talotyyppi to same as the rent
to_rplc = {'Kerrostalo yksiöt': 'Yksiöt', 'Kerrostalo kaksiot': 'Kaksiot', 'Kerrostalo kolmiot+': 'Kolmiot+'}
data_prices['Huoneluku'].replace(to_rplc, inplace=True) 


data_prices
# WE NEED TO CHECK if they have put some inconsistent data somewhere like 2023Q2* =/= 2023Q2

,Vuosineljännes,Huoneluku,Neliöhinta (EUR/m2),Lukumäärä,Posti,Kaupunginosa,Kaupunki
0,2009Q1,Yksiöt,4751.0,36,00100,Helsinkikeskusta-Etu-Töölö,Helsinki
1,2009Q1,Yksiöt,4561.0,8,00120,Punavuori-Bulevardi,Helsinki
2,2009Q1,Yksiöt,NaN,1,00130,Kaartinkaupunki,Helsinki
3,2009Q1,Yksiöt,5128.0,12,00140,Kaivopuisto-Ullanlinna,Helsinki
4,2009Q1,Yksiöt,4472.0,40,00150,Punavuori-Eira-Hernesaari,Helsinki
...,...,...,...,...,...,...,...
98251,2023Q2*,Kolmiot+,NaN,2,33850,Multisilta,Tampere
98252,2023Q2*,Kolmiot+,3615.0,7,33870,Vuores,Tampere
98254,2023Q2*,Kolmiot+,3883.0,8,33900,Härmälä-Rantaperkiö,Tampere
98261,2023Q2*,Kolmiot+,NaN,.,34240,Kämmenteisko,Tampere
